# Gray Matter CoreTech Sample data

Instructions:
Steps to compute measure score:
1. Select a measure
2. Filter records with diagnosis_codes that corresponds to the selected measure
Refer to the Diagnosis Codes table for diagnosis_codes for a given measure
3. Calculate comorbidity value for each row depending on set of comorbidity columns for the selected measure
Comorbidity value for a row can be calculated as count of ‘yes’ in corresponding comorbidity column set.Refer to the Comorbidity Columns table to find comorbidity columns for a given measure
4. Calculate lace score for each row in the dataset as sum of points for each LACE variable
Refer to the Lace Index table to find lace point for a given LACE variable
5. Select count of records as denominator
6. Select count of records with lace score > 9 as numerator
7. Calculate score for selected measure as numerator/denominator


# Initial Data wrangling

import pandas, read in csv
print data columns

In [64]:
import pandas as pd

data = pd.read_csv("SampleData2016.csv")
data.head()

,encounter_id,patient_nbr,race,gender,age,LengthofStay,ED_visits,Inpatient_visits,diagnosis_code,HistoryofPTCA,...,Urinarytractinfection,Otherinjuries,Skeletaldeformities,Posttraumaticosteoarthritis,Morbidobesity,Hypertension,Majorsymptomsabnormalities,HistoryofMechanicalVentilation,SleepApnea,OtherandUnspecifiedHeartDisease
0,2278392.0,8222157.0,Caucasian,Female,67.0,1.0,0.0,0.0,410.11,No,...,No,No,No,No,No,No,Yes,Yes,No,No
1,149190.0,55629189.0,Caucasian,Female,78.0,3.0,0.0,0.0,404.91,No,...,No,No,No,No,No,No,No,No,No,No
2,64410.0,86047875.0,AfricanAmerican,Female,86.0,2.0,5.0,1.0,CCS,No,...,No,Yes,No,No,Yes,No,No,Yes,No,Yes
3,500364.0,82442376.0,Caucasian,Male,65.0,2.0,1.0,0.0,402.01,No,...,No,Yes,No,No,No,No,No,No,No,Yes
4,16680.0,42519267.0,Caucasian,Male,60.0,1.0,0.0,0.0,402.11,No,...,No,Yes,No,No,Yes,No,No,No,No,Yes


In [54]:
columnNameList = data.columns.values.tolist()
print("data type: {}".format(type(data)))
print("data columns:")
for columnName in columnNameList:
    print(columnName)

data type: <class 'pandas.core.frame.DataFrame'>
data columns:
encounter_id
patient_nbr
race
gender
age
LengthofStay
ED_visits
Inpatient_visits
diagnosis_code
HistoryofPTCA
HistoryofCABG
Congestiveheartfailure
Acutecoronarysyndrome
Anteriormyocardialinfarction
Otherlocationofmyocardialinfarction
Anginapectorisoldmyocardialinfarction
Coronaryatherosclerosis
Valvularorrheumaticheartdisease
Specifiedarrhythmias
Historyofinfection
Metastaticcanceroracuteleukemia
Cancer
Diabetesmellitus(DM)orDMcomplications
Protein-caloriemalnutrition
Disordersoffluidelectrolyteacid-base
Irondeficiencyorotheranemiasandblooddisease
Dementiaorotherspecifiedbraindisorders
Hemiplegiaparaplegiaparalysisfunctionaldisability
Stroke
Cerebrovasculardisease
Vascularorcirculatorydisease
Chronicobstructivepulmonarydisease
Asthma
Pneumonia
End-stagerenaldiseaseordialysis
Renalfailure
Otherurinarytractdisorders
Decubitusulcerorchronicskinulcer
LungUpperDigestiveTractandOtherSevereCancers
LymphaticHeadandNeckBrainandOther

# Game plan

approach 1:
diagnosis codes, comorbidity values are like tables in rdb, could query to get values we want, then run program with values from those queries
approach 2:
quick and dirty, just put diagnosis codes, comorbidity values in csv, read them, and use them to run program

In [45]:
#get first row, look at types and values
row0 = data.iloc[0]
print("row0 type: {}".format(type(row0)))
for idx, element in enumerate(row0):
    print("idx: {} \tval: {} \ttype: {} \tcol: {}".format(idx, element, type(element), columnNameList[idx]))


row0 type: <class 'pandas.core.series.Series'>
idx: 0 	val: 2278392.0 	type: <class 'numpy.float64'> 	col: encounter_id
idx: 1 	val: 8222157.0 	type: <class 'numpy.float64'> 	col: patient_nbr
idx: 2 	val: Caucasian 	type: <class 'str'> 	col: race
idx: 3 	val: Female 	type: <class 'str'> 	col: gender
idx: 4 	val: 67.0 	type: <class 'numpy.float64'> 	col: age
idx: 5 	val: 1.0 	type: <class 'numpy.float64'> 	col: LengthofStay
idx: 6 	val: 0.0 	type: <class 'numpy.float64'> 	col: ED_visits
idx: 7 	val: 0.0 	type: <class 'numpy.float64'> 	col: Inpatient_visits
idx: 8 	val: 410.11 	type: <class 'str'> 	col: diagnosis_code
idx: 9 	val: No 	type: <class 'str'> 	col: HistoryofPTCA
idx: 10 	val: No 	type: <class 'str'> 	col: HistoryofCABG
idx: 11 	val: No 	type: <class 'str'> 	col: Congestiveheartfailure
idx: 12 	val: No 	type: <class 'str'> 	col: Acutecoronarysyndrome
idx: 13 	val: No 	type: <class 'str'> 	col: Anteriormyocardialinfarction
idx: 14 	val: No 	type: <class 'str'> 	col: Otherlocati

# Issue seen
what's up with column 39? LymphaticHeadandNeckBrainandOtherMajorCancers;BreastColorectalandotherCancersandTumors;OtherRespiratoryandHeartNeoplasms
why does it have semicolons in it? is it a catch-all for 3 types or is this a typo?

In [28]:
#verify that column name list length and row 0 are exactly equal:
if(len(columnNameList)!= len(row0)):
    raise ValueError("number of column labels not equal to number data columns ")
else:
    print("number of data columns = {}".format(len(columnNameList)))

number of data columns = 82


In [48]:
#pandas import mismatched header/data column test
badData = pd.read_csv("bad.csv")
print(badData.head())
print("shape: {}".format(badData.shape))
print("type: {}".format(type(badData)))
for element in badData:
    print("{} \ttype:{}".format(element, type(element)))

bdrow0 = badData.iloc[0]
print(len(bdrow0))
for element in bdrow0:
    print("{} \ttype:{}".format(element, type(element)))

     first_name last_name;greeting
Hans      Weeks              hello
shape: (1, 2)
type: <class 'pandas.core.frame.DataFrame'>
first_name 	type:<class 'str'>
last_name;greeting 	type:<class 'str'>
2
Weeks 	type:<class 'str'>
hello 	type:<class 'str'>


badData.csv should have 3 columns but instead it has 2! pandas does not throw any warnings about this mismatch

In [52]:
#at least this throws an exception! letting us know that size is unexpected
betterData = pd.read_csv("bad.csv", header=None)
print("betterData shape: {}".format(betterData.shape))


ParserError: Error tokenizing data. C error: Expected 2 fields in line 2, saw 3


In [53]:
#retry with sample data
dataframe2 = pd.read_csv("SampleData2016.csv", header=None)
print("dataframe2 shape: {}".format(dataframe2.shape))

dataframe2 shape: (1896, 82)


# ^ no data size errors, given data is good, end issue seen

# csv to relational db table

In [57]:
from sqlalchemy import create_engine

# create in memeory db
engine = create_engine('sqlite:///:memory:')

# store the data in 'engine' db as data_table
data.to_sql('data_table', engine)

#pandas sql query
result = pd.read_sql_query("SELECT * FROM data_table limit 1",engine)

print(result)



   index  encounter_id  patient_nbr       race  gender   age  LengthofStay  \
0      0     2278392.0    8222157.0  Caucasian  Female  67.0           1.0   

   ED_visits  Inpatient_visits diagnosis_code               ...                \
0        0.0               0.0         410.11               ...                 

  Urinarytractinfection Otherinjuries Skeletaldeformities  \
0                    No            No                  No   

  Posttraumaticosteoarthritis Morbidobesity Hypertension  \
0                          No            No           No   

  Majorsymptomsabnormalities HistoryofMechanicalVentilation SleepApnea  \
0                        Yes                            Yes         No   

  OtherandUnspecifiedHeartDisease  
0                              No  

[1 rows x 83 columns]


In [92]:
#getting measures without using rdb

#list of float
amiList = 410.00, 410.01, 410.10, 410.11, 410.20, 410.21, 410.30, 410.31, 410.40, 410.41, 410.50, 410.51, 410.60, 410.61, 410.70, 410.71, 410.80, 410.81, 410.90, 410.91

#list of string
amiList2 = []
for num in amiList:
    amiList2.append("%.2f" % num)
    
amiList = amiList2

print("len amiList: {}".format(len(amiList)))
print("amiList: {}".format(amiList))

#TODO add key string and value list for all measure/diagnosis code pairs
diagnosisCodeMap = {}
diagnosisCodeMap["ami"] = amiList

#filter dataframe on single diagnosis_code value:
is_410 = data['diagnosis_code'] == amiList[3]
data_410 = data[is_410]

#filter dataframe on list of diagnosis_code values:
data_ami = data[data.diagnosis_code.isin(amiList)]
print("rows in data where diagnosis code in amiList: {}".format(data_ami.shape))



len amiList: 20
amiList: ['410.00', '410.01', '410.10', '410.11', '410.20', '410.21', '410.30', '410.31', '410.40', '410.41', '410.50', '410.51', '410.60', '410.61', '410.70', '410.71', '410.80', '410.81', '410.90', '410.91']
rows in data where diagnosis code in amiList: (268, 82)


# Begin Comorbidity Functions

In [349]:
import pandas as pd

data = pd.read_csv("SampleData2016.csv")
data.head()

,encounter_id,patient_nbr,race,gender,age,LengthofStay,ED_visits,Inpatient_visits,diagnosis_code,HistoryofPTCA,...,Urinarytractinfection,Otherinjuries,Skeletaldeformities,Posttraumaticosteoarthritis,Morbidobesity,Hypertension,Majorsymptomsabnormalities,HistoryofMechanicalVentilation,SleepApnea,OtherandUnspecifiedHeartDisease
0,2278392.0,8222157.0,Caucasian,Female,67.0,1.0,0.0,0.0,410.11,No,...,No,No,No,No,No,No,Yes,Yes,No,No
1,149190.0,55629189.0,Caucasian,Female,78.0,3.0,0.0,0.0,404.91,No,...,No,No,No,No,No,No,No,No,No,No
2,64410.0,86047875.0,AfricanAmerican,Female,86.0,2.0,5.0,1.0,CCS,No,...,No,Yes,No,No,Yes,No,No,Yes,No,Yes
3,500364.0,82442376.0,Caucasian,Male,65.0,2.0,1.0,0.0,402.01,No,...,No,Yes,No,No,No,No,No,No,No,Yes
4,16680.0,42519267.0,Caucasian,Male,60.0,1.0,0.0,0.0,402.11,No,...,No,Yes,No,No,Yes,No,No,No,No,Yes


In [350]:
#function to read csv of column name=measure and followed by n rows of diagnosis codes that belong to that measure

#returns pair (key measure, value [list diagnosis codes])
def readDiagnosisCodeFromFile(filename):
    measureData = pd.read_csv(filename, dtype={0: str})#want dc as string even if it appears numerical
    
    measureDataColumns = measureData.columns.values.tolist()
    
    #assert that the list is length 1
    if len(measureDataColumns) > 1:
        raise ValueError("Diagnosis Code file {} contained too many columns, expected 1 column found {}"
                         .format(filname, len(measureDataColumns)))
    
    measure = measureDataColumns[0]
    diagnosisCodes = measureData[measure].values.tolist()
    
    return (measure, diagnosisCodes)

pair = readDiagnosisCodeFromFile("DiagnosisCode/AMI.csv")
pair

('AMI',
 ['410.00',
  '410.01',
  '410.10',
  '410.11',
  '410.20',
  '410.21',
  '410.30',
  '410.31',
  '410.40',
  '410.41',
  '410.50',
  '410.51',
  '410.60',
  '410.61',
  '410.70',
  '410.71',
  '410.80',
  '410.81',
  '410.90',
  '410.91'])

In [351]:
#function to get measure : diagnosis code pairs and build dictionary with them
from os import walk
from os import chdir

#returns dictionary, takes arg dir where measure csv's located
def buildMeasureToDiagnosisCodeDict(directory):
    
    measureDict = {}
    fileList = []
    
    #get filenames
    for (dirpath, dirname, filenames) in walk(directory):
        fileList.extend(filenames)
        break
        
    #cd to appropraite directory
    chdir(directory)
        
    for filename in fileList:
        (measure, diagnosisCodes) = readDiagnosisCodeFromFile(filename)
        measureDict[measure] = diagnosisCodes
        
    chdir("..")
        
    return measureDict

measureDict = buildMeasureToDiagnosisCodeDict("DiagnosisCode/")
measureDict

{'AMI': ['410.00',
  '410.01',
  '410.10',
  '410.11',
  '410.20',
  '410.21',
  '410.30',
  '410.31',
  '410.40',
  '410.41',
  '410.50',
  '410.51',
  '410.60',
  '410.61',
  '410.70',
  '410.71',
  '410.80',
  '410.81',
  '410.90',
  '410.91'],
 'COPD': ['491.21',
  '491.22',
  '491.8',
  '491.9',
  '492.8',
  '493.20',
  '493.21',
  '493.22',
  '496',
  '518.81',
  '518.82',
  '518.84',
  '799.1']}

In [352]:
#function that returns a dataframe of the rows with the diagnosis codes corresponding to each measure

def getDFRowsForMeasure(dataframe, measureDict, measure):
    diagnosisCodeList = measureDict[measure]
    data_ami = data[data.diagnosis_code.isin(diagnosisCodeList)]
    subdataframe = dataframe[dataframe.diagnosis_code.isin(diagnosisCodeList)]

    return subdataframe

amiDF = getDFRowsForMeasure(data, measureDict, "AMI")
amiDF

,encounter_id,patient_nbr,race,gender,age,LengthofStay,ED_visits,Inpatient_visits,diagnosis_code,HistoryofPTCA,...,Urinarytractinfection,Otherinjuries,Skeletaldeformities,Posttraumaticosteoarthritis,Morbidobesity,Hypertension,Majorsymptomsabnormalities,HistoryofMechanicalVentilation,SleepApnea,OtherandUnspecifiedHeartDisease
0,2278392.0,8222157.0,Caucasian,Female,67.0,1.0,0.0,0.0,410.11,No,...,No,No,No,No,No,No,Yes,Yes,No,No
27,248916.0,115196778.0,Caucasian,Female,89.0,2.0,2.0,0.0,410.11,No,...,Yes,Yes,No,No,No,No,No,No,Yes,Yes
30,253380.0,56480238.0,AfricanAmerican,Female,74.0,6.0,0.0,0.0,410.21,No,...,No,No,Yes,No,No,No,No,No,No,No
40,383430.0,80588529.0,Caucasian,Female,75.0,1.0,0.0,0.0,410.21,Yes,...,No,No,No,No,No,Yes,Yes,No,No,No
63,685086.0,24370299.0,Caucasian,Male,75.0,4.0,0.0,0.0,410.71,No,...,No,No,No,Yes,Yes,No,No,No,No,No
65,713268.0,58231314.0,?,Female,75.0,8.0,3.0,0.0,410.41,No,...,No,Yes,No,No,No,No,No,Yes,No,Yes
66,715086.0,3376278.0,Caucasian,Male,64.0,1.0,0.0,0.0,410.91,No,...,No,No,Yes,Yes,No,Yes,Yes,No,No,No
77,1050294.0,100113975.0,Caucasian,Female,80.0,3.0,1.0,0.0,410.51,No,...,Yes,No,No,No,No,No,No,No,Yes,No
78,1069716.0,34806888.0,Caucasian,Female,81.0,2.0,1.0,0.0,410.21,No,...,No,Yes,No,No,No,No,No,No,No,Yes
89,1157574.0,91181331.0,Caucasian,Male,73.0,4.0,1.0,1.0,410.01,No,...,No,No,No,No,No,Yes,No,Yes,No,No


In [353]:
#readDiagnosisCodeFromFile() also works for reading comorbidity
amiCM = readDiagnosisCodeFromFile("ComorbidityColumns/AMI.csv")

#buildMeasureToDiagnosisCodeDict() also works for comorbidity
comorbidityDict = buildMeasureToDiagnosisCodeDict("ComorbidityColumns/")
comorbidityDict

{'AMI': ['HistoryofPTCA',
  'HistoryofCABG',
  'Congestiveheartfailure',
  'Acutecoronarysyndrome',
  'Anteriormyocardialinfarction',
  'Otherlocationofmyocardialinfarction',
  'Anginapectorisoldmyocardialinfarction',
  'Coronaryatherosclerosis',
  'Valvularorrheumaticheartdisease',
  'Specifiedarrhythmias',
  'Historyofinfection',
  'Metastaticcanceroracuteleukemia',
  'Cancer',
  'Diabetesmellitus(DM)orDMcomplications',
  'Protein-caloriemalnutrition',
  'Disordersoffluidelectrolyteacid-base',
  'Irondeficiencyorotheranemiasandblooddisease',
  'Dementiaorotherspecifiedbraindisorders',
  'Hemiplegiaparaplegiaparalysisfunctionaldisability',
  'Stroke',
  'Cerebrovasculardisease',
  'Vascularorcirculatorydisease',
  'Chronicobstructivepulmonarydisease',
  'Asthma',
  'Pneumonia',
  'End-stagerenaldiseaseordialysis',
  'Renalfailure',
  'Otherurinarytractdisorders',
  'Decubitusulcerorchronicskinulcer'],
 'COPD': ['HistoryofMechanicalVentilation',
  'SleepApnea',
  'Respiratordependence/

In [354]:
#function to return dataframe cols assoc with measure

#given: measureDict, comorbidityDict, measureDF
def getComorbidityCols(measureDF, comorbidityDict, measure):
    cmList = comorbidityDict[measure]
    
    #drop measureDF cols not in cmList
    submeasureDF = measureDF[cmList]
    
    return submeasureDF
    
submeasureDF = getComorbidityCols(amiDF, comorbidityDict, "AMI")
submeasureDF


,HistoryofPTCA,HistoryofCABG,Congestiveheartfailure,Acutecoronarysyndrome,Anteriormyocardialinfarction,Otherlocationofmyocardialinfarction,Anginapectorisoldmyocardialinfarction,Coronaryatherosclerosis,Valvularorrheumaticheartdisease,Specifiedarrhythmias,...,Stroke,Cerebrovasculardisease,Vascularorcirculatorydisease,Chronicobstructivepulmonarydisease,Asthma,Pneumonia,End-stagerenaldiseaseordialysis,Renalfailure,Otherurinarytractdisorders,Decubitusulcerorchronicskinulcer
0,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
27,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
30,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
40,Yes,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
63,No,No,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
65,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
66,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
77,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
78,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
89,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No


In [355]:
#function to calculate comorbidity values

def getComorbidityValues(submeasureDF):
    #count occurences of values per row (i.e. yes=f1, no=f2)
    valueCount = submeasureDF.apply(pd.Series.value_counts, axis=1)
    #replace nan values with 0 (nan prevents transform to int)
    valueCount = valueCount.fillna(0)
    #transform float to int
    valueCount = valueCount.astype(int)
    return valueCount["Yes"]

comorbidityValuesAMI = getComorbidityValues(submeasureDF)
comorbidityValuesAMI

0       0
27      1
30      2
40      1
63      3
65      1
66      0
77      0
78      0
89      0
103     1
112     1
120     0
130     1
132     1
135     1
144     0
154     0
159     1
166     0
170     0
178     2
185     0
188     2
220     0
227     2
229     0
247     0
251     1
254     2
       ..
1720    2
1723    1
1742    1
1747    0
1754    1
1755    0
1764    0
1765    1
1766    0
1768    2
1771    0
1786    1
1787    1
1790    2
1795    1
1798    0
1816    0
1817    1
1819    2
1821    2
1827    1
1857    1
1861    3
1865    0
1868    1
1871    1
1874    1
1876    1
1879    0
1886    2
Name: Yes, Length: 268, dtype: int64

In [356]:
#add comorbidity value back on to measureDF
def addCMValuesToDF(measureDF, comorbidityValuesDF):
    #measureDF["ComorbidityValues"]=comorbidityValuesDF
    measureDF = measureDF.assign(ComorbidityValue=comorbidityValuesDF.values)

    return measureDF

amiDF = addCMValuesToDF(amiDF, comorbidityValuesAMI)
amiDF

,encounter_id,patient_nbr,race,gender,age,LengthofStay,ED_visits,Inpatient_visits,diagnosis_code,HistoryofPTCA,...,Otherinjuries,Skeletaldeformities,Posttraumaticosteoarthritis,Morbidobesity,Hypertension,Majorsymptomsabnormalities,HistoryofMechanicalVentilation,SleepApnea,OtherandUnspecifiedHeartDisease,ComorbidityValue
0,2278392.0,8222157.0,Caucasian,Female,67.0,1.0,0.0,0.0,410.11,No,...,No,No,No,No,No,Yes,Yes,No,No,0
27,248916.0,115196778.0,Caucasian,Female,89.0,2.0,2.0,0.0,410.11,No,...,Yes,No,No,No,No,No,No,Yes,Yes,1
30,253380.0,56480238.0,AfricanAmerican,Female,74.0,6.0,0.0,0.0,410.21,No,...,No,Yes,No,No,No,No,No,No,No,2
40,383430.0,80588529.0,Caucasian,Female,75.0,1.0,0.0,0.0,410.21,Yes,...,No,No,No,No,Yes,Yes,No,No,No,1
63,685086.0,24370299.0,Caucasian,Male,75.0,4.0,0.0,0.0,410.71,No,...,No,No,Yes,Yes,No,No,No,No,No,3
65,713268.0,58231314.0,?,Female,75.0,8.0,3.0,0.0,410.41,No,...,Yes,No,No,No,No,No,Yes,No,Yes,1
66,715086.0,3376278.0,Caucasian,Male,64.0,1.0,0.0,0.0,410.91,No,...,No,Yes,Yes,No,Yes,Yes,No,No,No,0
77,1050294.0,100113975.0,Caucasian,Female,80.0,3.0,1.0,0.0,410.51,No,...,No,No,No,No,No,No,No,Yes,No,0
78,1069716.0,34806888.0,Caucasian,Female,81.0,2.0,1.0,0.0,410.21,No,...,Yes,No,No,No,No,No,No,No,Yes,0
89,1157574.0,91181331.0,Caucasian,Male,73.0,4.0,1.0,1.0,410.01,No,...,No,No,No,No,Yes,No,Yes,No,No,0


# Begin LACE functions

In [357]:
#function to add emergency admission column (unncessary)
def createEmergencyAdmissionCol(dataframe):
    f = lambda n : 3 if n > 0 else 0
        
    EmergencyAdmissionCol = dataframe.Inpatient_visits.apply(f)
    dataframe = dataframe.assign(EmergencyAdmission=EmergencyAdmissionCol.values)
    return dataframe

#createEmergencyAdmissionCol(data)


In [358]:
#functions to map dataframe values to LACE scores

def lengthOfStayValueToLScore(n):
    if n<1:
        return 0
    elif n>=1 and n<=4:
        return n
    elif n>4 and n<=6:
        return 4
    elif n>=7 and n<=13:
        return 5
    else:
        return 7

def acuteAdmissionValueToAScore(n):
    if n>0:
        return 3
    else:
        return 0

def comorbidityValueToCScore(n):
    if n < 0:
        #this case shouldn't really be possible
        return 0
    elif n <= 3:
        return n
    else:
        return 5

def edVisitValueToEScore(n):
    if n < 0:
        return 0
    elif n <= 4:
        return n
    else:
        return 4

#test
l1 = []
for n in range(-1,17):
    l1.append((n,lengthOfStayValueToLScore(n)))
l1

l2=[]
for valuestr in range(-1,3):
    l2.append((valuestr,acuteAdmissionValueToAScore(valuestr)))   
l2

l3=[]
for n in range(-1,7):
    l3.append((n, comorbidityValueToCScore(n)))
l3

l4=[]
for n in range(-1, 6):
    l4.append((n, edVisitValueToEScore(n)))
l4



[(-1, 0), (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 4)]

In [359]:
#string equality test
import copy

s1 = "hi"
s2 = copy.copy(s1)
s1==s2
print("id s1:{}, id s2:{}".format(id(s1),id(s2)))

id s1:4389781552, id s2:4389781552


In [365]:
#functions to calculate LACE scores as individual elements

def calculateLScore(measureDF):
    
    lScore = measureDF.LengthofStay
    lScore = lScore.astype(int)
    lScore = lScore.apply(lengthOfStayValueToLScore)

    return lScore

def calculateAScore(measureDF):
    
    #missing this data column
    aScore = measureDF.Inpatient_visits
    aScore = aScore.apply(acuteAdmissionValueToAScore)
    
    return aScore

def calculateCScore(measureDF):
    
    cScore = measureDF.ComorbidityValue
    cScore = cScore.apply(comorbidityValueToCScore)
    
    return cScore

def calculateEScore(measureDF):
    
    eScore = measureDF.ED_visits
    eScore = eScore.astype(int)
    eScore = eScore.apply(edVisitValueToEScore)
    
    return eScore

calculateLScore(amiDF)
calculateAScore(amiDF)
calculateCScore(amiDF)
calculateEScore(amiDF)

0       0
27      2
30      0
40      0
63      0
65      3
66      0
77      1
78      1
89      1
103     0
112     1
120     4
130     0
132     0
135     0
144     1
154     1
159     1
166     0
170     1
178     0
185     3
188     3
220     2
227     0
229     4
247     1
251     2
254     4
       ..
1720    0
1723    1
1742    1
1747    1
1754    1
1755    0
1764    0
1765    4
1766    4
1768    0
1771    4
1786    0
1787    1
1790    1
1795    0
1798    0
1816    1
1817    1
1819    0
1821    2
1827    4
1857    0
1861    4
1865    1
1868    0
1871    0
1874    0
1876    0
1879    0
1886    0
Name: ED_visits, Length: 268, dtype: int64

In [366]:
#function to add L,A,C,E rows to dataframe

def calculateLACEColumns(measureDF):
    
    lScoreCol = calculateLScore(measureDF)
    
    aScoreCol = calculateAScore(measureDF)
    
    cScoreCol = calculateCScore(measureDF)
    
    eScoreCol = calculateEScore(measureDF)
    
#     print("{}\n{}\n{}\n{}".format(lScoreCol, aScoreCol, cScoreCol, eScoreCol))
    measureDF = measureDF.assign(LengthOfStayScore=lScoreCol.values)
    
    measureDF = measureDF.assign(AcuteAdmiisionScore=aScoreCol.values)
    
    measureDF = measureDF.assign(ComorbidityScore=cScoreCol.values)
    
    measureDF = measureDF.assign(EDVisitScore=eScoreCol.values)
    
    return measureDF
    
amiDF = calculateLACEColumns(amiDF)
amiDF    

,encounter_id,patient_nbr,race,gender,age,LengthofStay,ED_visits,Inpatient_visits,diagnosis_code,HistoryofPTCA,...,Majorsymptomsabnormalities,HistoryofMechanicalVentilation,SleepApnea,OtherandUnspecifiedHeartDisease,ComorbidityValue,LengthOfStayScore,AcuteAdmiisionScore,ComorbidityScore,EDVisitScore,LACEScore
0,2278392.0,8222157.0,Caucasian,Female,67.0,1.0,0.0,0.0,410.11,No,...,Yes,Yes,No,No,0,1,0,0,0,1
27,248916.0,115196778.0,Caucasian,Female,89.0,2.0,2.0,0.0,410.11,No,...,No,No,Yes,Yes,1,2,0,1,2,5
30,253380.0,56480238.0,AfricanAmerican,Female,74.0,6.0,0.0,0.0,410.21,No,...,No,No,No,No,2,4,0,2,0,6
40,383430.0,80588529.0,Caucasian,Female,75.0,1.0,0.0,0.0,410.21,Yes,...,Yes,No,No,No,1,1,0,1,0,2
63,685086.0,24370299.0,Caucasian,Male,75.0,4.0,0.0,0.0,410.71,No,...,No,No,No,No,3,4,0,3,0,7
65,713268.0,58231314.0,?,Female,75.0,8.0,3.0,0.0,410.41,No,...,No,Yes,No,Yes,1,5,0,1,3,9
66,715086.0,3376278.0,Caucasian,Male,64.0,1.0,0.0,0.0,410.91,No,...,Yes,No,No,No,0,1,0,0,0,1
77,1050294.0,100113975.0,Caucasian,Female,80.0,3.0,1.0,0.0,410.51,No,...,No,No,Yes,No,0,3,0,0,1,4
78,1069716.0,34806888.0,Caucasian,Female,81.0,2.0,1.0,0.0,410.21,No,...,No,No,No,Yes,0,2,0,0,1,3
89,1157574.0,91181331.0,Caucasian,Male,73.0,4.0,1.0,1.0,410.01,No,...,No,Yes,No,No,0,4,3,0,1,8


In [367]:
#function to sum LACE final score

def calculateLACEScore(measureDF):
    columns = ["LengthOfStayScore", "AcuteAdmiisionScore", "ComorbidityScore", "EDVisitScore"]
    laceColumns = measureDF[columns]
    
    laceScore = laceColumns.sum(axis=1)
    
    measureDF = measureDF.assign(LACEScore=laceScore.values)
    return measureDF

    
    
amiDF = calculateLACEScore(amiDF)
amiDF

,encounter_id,patient_nbr,race,gender,age,LengthofStay,ED_visits,Inpatient_visits,diagnosis_code,HistoryofPTCA,...,Majorsymptomsabnormalities,HistoryofMechanicalVentilation,SleepApnea,OtherandUnspecifiedHeartDisease,ComorbidityValue,LengthOfStayScore,AcuteAdmiisionScore,ComorbidityScore,EDVisitScore,LACEScore
0,2278392.0,8222157.0,Caucasian,Female,67.0,1.0,0.0,0.0,410.11,No,...,Yes,Yes,No,No,0,1,0,0,0,1
27,248916.0,115196778.0,Caucasian,Female,89.0,2.0,2.0,0.0,410.11,No,...,No,No,Yes,Yes,1,2,0,1,2,5
30,253380.0,56480238.0,AfricanAmerican,Female,74.0,6.0,0.0,0.0,410.21,No,...,No,No,No,No,2,4,0,2,0,6
40,383430.0,80588529.0,Caucasian,Female,75.0,1.0,0.0,0.0,410.21,Yes,...,Yes,No,No,No,1,1,0,1,0,2
63,685086.0,24370299.0,Caucasian,Male,75.0,4.0,0.0,0.0,410.71,No,...,No,No,No,No,3,4,0,3,0,7
65,713268.0,58231314.0,?,Female,75.0,8.0,3.0,0.0,410.41,No,...,No,Yes,No,Yes,1,5,0,1,3,9
66,715086.0,3376278.0,Caucasian,Male,64.0,1.0,0.0,0.0,410.91,No,...,Yes,No,No,No,0,1,0,0,0,1
77,1050294.0,100113975.0,Caucasian,Female,80.0,3.0,1.0,0.0,410.51,No,...,No,No,Yes,No,0,3,0,0,1,4
78,1069716.0,34806888.0,Caucasian,Female,81.0,2.0,1.0,0.0,410.21,No,...,No,No,No,Yes,0,2,0,0,1,3
89,1157574.0,91181331.0,Caucasian,Male,73.0,4.0,1.0,1.0,410.01,No,...,No,Yes,No,No,0,4,3,0,1,8


In [368]:
#summing dataframe rows and columns example
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)

df.apply(pd.Series.value_counts, axis=1)

,1,2,3,4
0,1.0,NaN,1.0,NaN
1,NaN,1.0,NaN,1.0


In [369]:
#function to calculate final measure score
#measure score = (count LACE score>9)/(count measure rows)
def calculateMeasureScore(measureName, measureDF):
    LACEScores=amiDF.LACEScore
    
    numeratorCol = amiDF[amiDF.LACEScore > 9]
    numerator = float(len(numeratorCol))
    
    denominator = float(len(amiDF))
    
    print("n={}, d={}".format(numerator, denominator))
    return numerator/denominator
    
calculateMeasureScore("ami", amiDF)

n=45.0, d=268.0


0.16791044776119404